In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

     |████████████████████████████████| 6.4 MB 36.9 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 46 kB 4.3 MB/s 


In [8]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(11,df.shape[0]-1):
        sugg = 0
        if df[each+1][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-10][3],
                df[each-9][3],
                df[each-8][3],
                df[each-7][3],
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                df[each][3], 
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object)
  scaled["sugg"] = last_column
  return scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df)
      df = row_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df      

In [9]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")

symbols = get_crypto_syms()
dfs = []
for symbol in symbols:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
data = process(dfs)
data.to_csv("1d_10pastcandle_newversion.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data


1 Failed download:
- XNO-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-AUD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- VAL-CAD: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- ICP-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XNO-EUR: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- OXEN-EUR: No data found for this date range, symbol may be delisted
Passing.

,0,1,2,3,4,5,6,7,8,9,10,sugg
0,0.806330,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
1,0.320420,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
4,0.445989,1.000000,0.792717,0.601163,0.483424,0.402642,0.034734,0.006506,0.195524,0.140697,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1760289,0.156831,0.949573,0.501762,0.824854,0.691509,1.000000,0.756873,0.793887,0.096352,0.288714,0.000000,0.0
1760290,0.962779,0.632245,0.870723,0.772300,1.000000,0.820545,0.847866,0.333008,0.474993,0.261890,0.000000,0.0
1760291,0.632245,0.870723,0.772300,1.000000,0.820545,0.847866,0.333008,0.474993,0.261890,0.000000,0.110323,0.0
1760292,0.903385,0.829828,1.000000,0.865884,0.886302,0.501522,0.607635,0.448372,0.252648,0.335097,0.000000,0.0


In [ ]:
symbols = read_syms_from_txt()
len(symbols)
pieces = 15
new_arrays = np.array_split(symbols, pieces)
for i in new_arrays:
  dfs = []
  for x in i:
        data = yf.download(x,period="MAX",interval="1d",progress=False)
        if data.empty :
             print("Passing...")
        else:
               dfs.append(data)
  ndfs = dfs
  dfs = []
  for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
  data = process(dfs)
  data.to_csv(f"{x}.csv")         

In [3]:
address = "/content/gdrive/MyDrive/Colab Files/past10candelclose_allsymtxt_1d.csv"
from google.colab import drive
drive.mount("/content/gdrive/")
data = pd.read_csv(address)
data = data.drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

Mounted at /content/gdrive/
(10294971, 11) (10294971,)
(1143886, 11) (1143886,)


,0,1,2,3,4,5,6,7,8,9,10,sugg
0,0.189842,0.433805,0.565362,1.000000,0.926728,0.875104,0.860949,0.358867,0.230641,0.000000,0.086594,0.0
1,0.433805,0.565362,1.000000,0.926728,0.875104,0.860949,0.358867,0.230641,0.000000,0.086594,0.034138,0.0
2,0.651303,1.000000,0.941216,0.899800,0.888444,0.485638,0.382765,0.197729,0.267201,0.225117,0.000000,0.0
3,1.000000,0.944758,0.905838,0.895166,0.516635,0.419962,0.246077,0.311362,0.271814,0.060264,0.000000,1.0
4,1.000000,0.958804,0.947508,0.546844,0.444518,0.260465,0.329568,0.287708,0.063787,0.000000,0.136213,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11438852,0.272726,0.000000,0.090903,0.727274,1.000000,1.000000,1.000000,1.000000,1.000000,0.909089,0.909089,1.0
11438853,0.000000,0.071424,0.571431,0.785715,0.785715,0.785715,0.785715,0.785715,0.714285,0.714285,1.000000,0.0
11438854,0.000000,0.538466,0.769233,0.769233,0.769233,0.769233,0.769233,0.692308,0.692308,1.000000,1.000000,0.0
11438855,0.000000,0.500000,0.500000,0.500000,0.500000,0.500000,0.333328,0.333328,1.000000,1.000000,0.000000,0.0


In [11]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=0.00146,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")

model.add(Dense(5000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(2500, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(500, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(25, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 5000)              60000     
                                                                 
 dense_22 (Dense)            (None, 2500)              12502500  
                                                                 
 dense_23 (Dense)            (None, 1000)              2501000   
                                                                 
 dense_24 (Dense)            (None, 500)               500500    
                                                                 
 dense_25 (Dense)            (None, 100)               50100     
                                                                 
 dense_26 (Dense)            (None, 25)                2525      
                                                                 
 dense_27 (Dense)            (None, 1)                

In [ ]:
model.fit(xTrain,yTrain,epochs=100,batch_size=20000,validation_data=(xTest,yTest))
model.evaluate(xTest,yTest)
model.save(f"{model.evaluate(xTest,yTest)}.h5")

Epoch 1/100
80/80 [==============================] - 41s 511ms/step - loss: 0.5458 - accuracy: 0.7168 - val_loss: 0.5120 - val_accuracy: 0.7490
Epoch 2/100
80/80 [==============================] - 43s 535ms/step - loss: 0.5109 - accuracy: 0.7491 - val_loss: 0.5081 - val_accuracy: 0.7499
Epoch 3/100
80/80 [==============================] - 44s 544ms/step - loss: 0.5070 - accuracy: 0.7508 - val_loss: 0.5057 - val_accuracy: 0.7510
Epoch 4/100
25/80 [========>.....................] - ETA: 28s - loss: 0.5039 - accuracy: 0.7527

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

In [ ]:
import os
files = os.listdir("/content/gdrive/MyDrive/Colab Files/")

data = pd.DataFrame()
for file in files:
  address = f"/content/gdrive/MyDrive/Colab Files/{file}"
  df = pd.read_csv(address)
  data = pd.concat([data,df])